## ThaiSum Dataset

- **`train`** → 358,868 ข่าว → ใช้ Train Model
- **`validation`** → 11,000 ข่าว → ใช้ตรวจสอบระหว่าง Train
- **`test`** → 11,000 ข่าว → ใช้ Test Model 
- **`valid`** → 11,000 ข่าว → อีกชุดสำหรับตรวจสอบ  

### ข้อมูลในแต่ละข่าว
- **`title`** → หัวข้อข่าว  
- **`body`** → เนื้อหาข่าวเต็ม  
- **`summary`** → สรุปสั้น ๆ ของข่าว (คำตอบที่ Model ต้องเรียนรู้)  
- **`tags`** → คำสำคัญ/หมวดหมู่  
- **`url`** → Link ต้นฉบับ  


In [1]:
from datasets import load_dataset

dataset = load_dataset("pythainlp/thaisum")

print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 358868
    })
    validation: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
    test: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
    valid: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
})
{'title': ' วิษณุ ยันโรดแม็ปเดิม ตอบไม่ถูกเวลาเลือกตั้ง ต้องรอ รธน.ประกาศใช้', 'body': 'เมื่อวันที่ 6 ม.ค.60 ที่ทำเนียบรัฐบาล นายวิษณุ เครืองาม รองนายกรัฐมนตรี กล่าวถึงกรณี ที่ นายสุรชัย เลี้ยงบุญเลิศชัย รองประธานสภานิติบัญญัติแห่งชาติ (สนช.) ออกมาระบุว่า การเลือกตั้งจะถูกเลื่อนออกไปถึงปี 2561 ว่า ขอให้ไปสอบถามกับ สนช. แต่เชื่อว่าคงไม่กล้าพูดอีก เพราะทำให้คนเข้าใจผิด ซึ่งที่ สนช.พูดเนื่องจากผูกกับกฎหมายของกรรมการร่างรัฐธรรมนูญ(กรธ.) ตนจึงไม่ขอวิพากษ์วิจารณ์ แต่รัฐบาลยืนยันว่ายังเดิ

## mT5 Input requirement (ใช้ Tokenizer)
 
คือลองใส่ข้อความเข้า `AutoTokenizer` ของโมเดล → จะได้ dictionary ที่บอก keys ที่จำเป็น

- **`input_ids`** → ข้อความถูกแปลงเป็นตัวเลข เพื่อให้โมเดลเข้าใจได้  
- **`attention_mask`** → บอกว่าตัวเลขไหนเป็นข้อมูลจริง (1) หรือช่องว่าง (0)  


In [2]:
from transformers import AutoTokenizer

MODEL_NAME = "google/mt5-small" 
# Load tokenizer ของ mT5
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)

text = "Hello, world!"
inputs = tokenizer(text, return_tensors="pt") # คืนค่าข้อมูลเป็น Tensor ของ PyTorch 

print("Keys in inputs dictionary:", inputs.keys())

c:\Project\thai-summarization\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Keys in inputs dictionary: KeysView({'input_ids': tensor([[30273,   261,  4836,   309,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])})


## Token Length Statistics (Labels: Summaries)

- **Maximum token length:** `520`  
  - หมายความว่า summary ที่ **ยาวที่สุด** ใน dataset มีความยาว 520 tokens หลังจาก tokenize  

- **95th percentile (p95):** `79.0`  
  - หมายความว่า **95%** ของ summaries มีความยาว **≤ 79 tokens**  

- **99th percentile (p99):** `122.0`  
  - หมายความว่า **99%** ของ summaries มีความยาว **≤ 122 tokens**  

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

MODEL_NAME = "google/mt5-small"
# Load tokenizer ของ mT5
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)

dataset = load_dataset("pythainlp/thaisum")

# เก็บความยาว (จำนวน tokens) ของ label "summary" แต่ละตัว
summary_token_lengths = []

# loop ทีละตัวใน train set
for example in dataset["train"]:
    summary_text = example["summary"]  # ข้อความสรุป (label)
    tokenized_summary = tokenizer(summary_text)  # แปลงข้อความเป็น tokens
    num_tokens = len(tokenized_summary["input_ids"])  # นับจำนวน tokens
    summary_token_lengths.append(num_tokens)  # เก็บค่าไว้ใน list

max_length = max(summary_token_lengths)
p95_length = np.percentile(summary_token_lengths, 95)
p99_length = np.percentile(summary_token_lengths, 99)

print("Maximum token length:", max_length)
print("95 percent length: ", p95_length)
print("99 percent length: ", p99_length)

Maximum token length: 520
95 percent length:  79.0
99 percent length:  122.0


## ThaiSum Preprocessing Code

- โหลด ThaiSum dataset และลบ split ที่ซ้ำ  
- ตรวจสอบจำนวนข้อมูลและดูตัวอย่าง  
- แปลงข่าวเต็ม (`body`) → `input_ids` และสรุป (`summary`) → `labels`  
- จำกัดความยาว (body ≤ 512, summary ≤ 128)  
- ลบ column ที่ไม่ใช้ เหลือ input/labels พร้อม train mT5  

### Output หลัง Preprocessing
- **`input_ids`** → tokenized body (ข่าวเต็ม)  
- **`attention_mask`** → บอก padding (ตำแหน่งไหนควรถูก ignore)  
- **`labels`** → tokenized summary (สรุปข่าว)  


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer

MODEL_NAME = "google/mt5-small"
# Load tokenizer ของ mT5
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)

def load_thaisum():
    dataset = load_dataset("pythainlp/thaisum")
    # ลบ split 'valid' ที่ซ้ำกับ 'validation'
    # dataset ที่ได้จะมี 3 split : train, test, valid
    if "valid" in dataset:
        dataset.pop("valid")
    return dataset


def preprocess_dataset(dataset, tokenizer, max_input=512, max_target=128):
    # แปลง dataset ให้อยู่ในรูปแบบที่ใช้กับ Seq2SeqTrainer ได้
    # input_ids / attention_mask สำหรับ encoder
    # labels สำหรับ decoder
    def tokenize_fn(batch):
        # แปลงข่าวเต็ม (body) เป็น input_ids
        model_inputs = tokenizer(batch["body"], max_length=max_input, truncation=True)
        # แปลงสรุป (summary) เป็น labels
        labels = tokenizer(batch["summary"], max_length=max_target, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    # return dataset ที่มีเฉพาะ field : input_ids, attention_mask, labels
    return dataset.map(
        tokenize_fn,
        batched=True,
        remove_columns=["title", "body", "summary", "tags", "url", "type"],
    )


if __name__ == "__main__":
    dataset = load_thaisum()
    tokenized_dataset = preprocess_dataset(dataset, tokenizer)
    print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 358868
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11000
    })
})


## TrainingArguments

In [5]:
from transformers import TrainingArguments

help(TrainingArguments)

Help on class TrainingArguments in module transformers.training_args:

class TrainingArguments(builtins.object)
 |  TrainingArguments(
 |      output_dir: Optional[str] = None,
 |      overwrite_output_dir: bool = False,
 |      do_train: bool = False,
 |      do_eval: bool = False,
 |      do_predict: bool = False,
 |      eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no',
 |      prediction_loss_only: bool = False,
 |      per_device_train_batch_size: int = 8,
 |      per_device_eval_batch_size: int = 8,
 |      per_gpu_train_batch_size: Optional[int] = None,
 |      per_gpu_eval_batch_size: Optional[int] = None,
 |      gradient_accumulation_steps: int = 1,
 |      eval_accumulation_steps: Optional[int] = None,
 |      eval_delay: Optional[float] = 0,
 |      torch_empty_cache_steps: Optional[int] = None,
 |      learning_rate: float = 5e-05,
 |      weight_decay: float = 0.0,
 |      adam_beta1: float = 0.9,
 |      adam_beta2: float = 0.999,
 |      ada

## Quick Train Test Code

- โหลด **ThaiSum dataset** และ preprocess ด้วย `mT5 tokenizer`  
- โหลด **mT5 model (AutoModelForSeq2SeqLM)** สำหรับงานสรุปข่าว  
- ใช้ **DataCollatorForSeq2Seq** → จัด batch ให้อัตโนมัติ (padding + attention mask)  
- ตั้งค่า **TrainingArguments** → train แค่ 1 step (batch size=2)  
- ใช้ **Trainer** → จัดการ train loop ให้อัตโนมัติ  

In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "src"))

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
)
from preprocess import load_thaisum, preprocess_dataset, MODEL_NAME


def quick_train_test():
    # Load tokenizer + dataset
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)
    dataset = load_thaisum()
    tokenized_dataset = preprocess_dataset(dataset, tokenizer)

    # Load Model mT5 เเบบ Seq2Seq (สรุป/แปลภาษา)
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

    # สร้าง data collator : ตัวช่วยจัด batch (padding ให้อัตโนมัติ)
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

    # training arguments (train แค่ 1 step)
    training_args = TrainingArguments(
        output_dir="./test-output",  # ที่เก็บ Results
        per_device_train_batch_size=2,  # batch size = 2
        num_train_epochs=1,  # 1 epoch
        max_steps=1,  # train : 1 step
        logging_steps=1,  # log ทุก step
    )

    # ใช้ Trainer จัดการ train loop ให้
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"].select(
            range(10)
        ),  # ใช้ train : 10 ตัวอย่าง
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

    # 6) start training
    trainer.train()


if __name__ == "__main__":
    quick_train_test()

c:\Project\thai-summarization\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/11000 [00:00<?, ? examples/s]

C:\Users\ryuro\AppData\Local\Temp\ipykernel_29008\498214937.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1,13.231500


## Pre-Check

In [2]:
import torch

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
print(
    "MPS available (Mac):",
    getattr(torch.backends, "mps", None) and torch.backends.mps.is_available(),
)
print(
    "Device used by default:",
    torch.device(
        "cuda"
        if torch.cuda.is_available()
        else (
            "mps"
            if getattr(torch.backends, "mps", None)
            and torch.backends.mps.is_available()
            else "cpu"
        )
    ),
)

Torch: 2.8.0+cu126
CUDA available: True
CUDA version: 12.6
GPU name: NVIDIA GeForce RTX 3060
MPS available (Mac): False
Device used by default: cuda
